# Algorytmy Tekstowe 2019/2020

# Laboratorium 7

# Autor - Łukasz Jezapkowicz

# 1. Zaimplementuj algorytm wyszukiwania wyrażeń regularnych. Wyrażenie może zawierać:

   # - litery, cyfry i spacje - traktować literalnie 
   # - kropki - reprezentuje dowolny znak 
   # - operatory: gwiazdkę - 0 lub więcej powtórzeń poprzedniego symbolu, plus - 1 lub więcej powtórzeń, pytajnik - 0 lub 1 powtórzenie
   # - nawiasy - na potrzeby operatorów gwiazdki, plusa i pytajnika zawartość nawiasów jest pojedynczym symbolem; nawiasy mogą być zagnieżdżone; nawiasy po których nie występuje żaden z wymienionych operatorów nie mają skutku
   # - klasy znaków (albo dowolna liczba znaków wymieniona w nawiasach kwadratowych, albo oznaczenie klasy, typu \d)
   # - Możesz założyć, że wprowadzone wyrażenie jest poprawne.

W celu rozwiązania powyższego problemu będę budować automat skończony związany z danym wyrażeniem regularnym. Przy pomocy zbudowanego automatu będę mógł sprawdzać zgodność słowa z danym wyrażeniem regularnym. Wyniki będę porównywał z wbudowanym w Pythona regex'em.  

Pierwszym krokiem, który musimy wykonać jest transformacja danego wyrażenia regularnego z postaci infiksowej do postaci postfiksowej. Algorytmem, który pozwala wykonać taką czynność jest algorytm Shunting-Yard (https://en.wikipedia.org/wiki/Shunting-yard_algorithm).  
Drugą rzeczą jaką należy wykonać jest dodanie specjalnego znaku konkatenacji (zazwyczaj stosuje się do tego '.', ale tutaj używamy jej w innym celu, użyłem więc znaku '@').  
Należy również przekształcić nawiasy kwadratowe oraz klasy znaków na odpowiednie alternatywy ($[abc]\rightarrow(a|b|c)$).  
Funkcje dokonujące zamiany wyrażenia z postaci infiksowej do postaci postfiksowej widoczne są poniżej.

In [240]:
import string

# funkcja dodająca specjalny znak konkatenacji '@', potrzebny przy tworzeniu automatu skończonego
def add_concat_sign(infix):
    result = ""
    
    # znaki specjalne wyznaczające jak dodawać znak konkatenacji, None - początek wzorca 
    concat_from = ['(', '|', None]
    concat_to = ['*', '+', '?', ')', '|']
    previous_char = None
    
    # dodawanie znaku konkatenacji
    for char in infix:
        if char not in concat_to and previous_char not in concat_from:
            result += '@'
        result += char
        previous_char = char

    return result

# funkcja zamieniająca wyrażenia w nawiasach kwadratowych na odpowiednie alternatywy
def parse_square(infix):
    new_infix = ""
    
    i = 0
    while i < len(infix):
        c = infix[i]
        if c != '[':
            new_infix += c
        else:
            parsed = '('
            i += 1
            while infix[i] != ']':
                parsed += infix[i]
                if infix[i+1] != ']':
                    parsed += '|'
                i += 1
            parsed += ')'
            new_infix += parsed
        i += 1
    
    return new_infix

# funkcja zamieniająca klasy \d, \w oraz \s na odpowiednie alternatywy
def parse_backslashes(infix):
    new_infix = ""
    
    i = 0
    while i < len(infix):
        c = infix[i]
        if c != '\\':
            new_infix += c
        else:
            i += 1
            if infix[i] == 'd':
                new_infix += '(0|1|2|3|4|5|6|7|8|9)'
            elif infix[i] == 'w':
                parsed = '('
                for c in string.ascii_letters:
                    parsed += c + '|'
                
                parsed = parsed[:-1]
                parsed += ')'
                new_infix += parsed
            elif infix[i] == 's':
                parsed = '('
                for c in string.whitespace:
                    parsed += c + '|'
                
                parsed = parsed[:-1]
                parsed += ')'
                new_infix += parsed
            
        i += 1
    
    return new_infix

# funkcja zamieniająca infiksowy regex na postfixowy regex, wykorzystuje algorytm Shunting-Yard
def to_postfix(infix, print_regexes = False):
    # Curly braces = dictionary
    # * = zero albo więcej
    # + = raz albo więcej
    # ? = zero albo raz
    # @ = konkatenacja
    # | = alternatywa
    
    # określenie znaków specjalnych i ich kolejności (ważności)
    specials = {'?': 70, '+': 60, '*': 50, '@': 40, '|': 30}

    postfix = ""
    stack = ""
    
    # dodajemy znaki konkatenacji, zmieniamy kwadratowe nawiasy i klasy typu \d
    if print_regexes:
        print('Basic infix regex form: ' + infix)
    infix = parse_square(infix)
    infix = parse_backslashes(infix)
    infix = add_concat_sign(infix)
    if print_regexes:
        print('Parsed infix regex form: ' + infix)
 
    for c in infix:
        if c == '(':
            stack = stack + c
        elif c == ')':
            while stack[-1] != '(':
                postfix, stack = postfix + stack[-1], stack[:-1]
            # Usuwamy '(' ze stosu
            stack = stack[:-1]
        elif c in specials:
            while stack and specials.get(c, 0) <= specials.get(stack[-1], 0):
                postfix, stack = postfix + stack[-1], stack[:-1]
            stack = stack + c
        else:
            postfix = postfix + c

    while stack:
        postfix, stack = postfix + stack[-1], stack[:-1]
    
    if print_regexes:
        print('Parsed postfix regex form: ' + postfix)
        
    return postfix

Poniżej przedstawiam przykładowe działanie tych funkcji.

In [241]:
print("Postfix of abcdef is " + to_postfix("abcdef"))
print("Postfix of (ab|cd)* is " + to_postfix("(ab|cd)*"))
print("Postfix of (a|b)*a+b? is " + to_postfix("(a|b)*a+b?"))
print("Postfix of [abcd]*a+b\d? is " + to_postfix("[abcd]*a+b\d"))

Postfix of abcdef is ab@c@d@e@f@
Postfix of (ab|cd)* is ab@cd@|*
Postfix of (a|b)*a+b? is ab|*a+@b?@
Postfix of [abcd]*a+b\d? is ab|c|d|*a+@b@01|2|3|4|5|6|7|8|9|@


In [244]:
tmp = to_postfix('\d*.\d+', True)

Basic infix regex form: \d*.\d+
Parsed infix regex form: (0|1|2|3|4|5|6|7|8|9)*@.@(0|1|2|3|4|5|6|7|8|9)+
Parsed postfix regex form: 01|2|3|4|5|6|7|8|9|*.@01|2|3|4|5|6|7|8|9|+@


Następnym krokiem jest stworzenie automatu skończonego. W tym celu stosuje algorytm Thompsona budowania automatu skończonego dla danego wyrażenia regularnego (https://en.wikipedia.org/wiki/Thompson%27s_construction). 

In [245]:
# Klasa reprezentująca stan automatu z nazwą 'label' oraz dwoma krawędziami
# None oznacza tutaj stan z krawędzia 'epsilon'
class state: 
    label = None
    edge1 = None
    edge2 = None

# Klasa symbolizująca NFA (niedeterministyczny automat skończony)
class nfa:
    def __init__(self, initial, accept):
        self.initial = initial
        self.accept = accept

# funkcja zamieniająca wyrażenie regularne na NFA
# tworzymy nowe automaty i je łączymy, tak jak opisuje algorytm Thompsona
def compile(postfix):
    # pomocniczy stos, na koniec procedury powinien zawierać tylko 1 element (nasz automat)
    nfaStack = []

    for c in postfix:
        if c == '?':
            # Zdejmujemy automat ze stosu
            nfa1 = nfaStack.pop()

            # Tworzymy stan początkowy i końcowy dla nowego automatu
            initial = state()
            accept = state()
            
            # Łączymy stan początkowy nowego automatu ze stanem początkowym nfa1
            initial.edge1 = nfa1.initial
            initial.edge2 = accept
            
            # Łączymy stan akceptujący nfa1 ze stanem akceptującym nowego automatu
            nfa1.accept.edge1 = accept

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(initial, accept))
            
        elif c == '+':
            # Zdejmujemy automat ze stosu
            nfa1 = nfaStack.pop()

            # Tworzymy stan początkowy i końcowy dla nowego automatu
            initial = state()
            accept = state()
            
            # Łączymy krawędź nowego stanu początkowego ze stanem początkowym nfa1
            initial.edge1 = nfa1.initial
            
            # Łączymy krawędzie stanu akceptującego nfa1 ze stanem początkowym nfa1 oraz nowym stanem akceptującym
            nfa1.accept.edge1 = nfa1.initial
            nfa1.accept.edge2 = accept

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(initial, accept))

        elif c == '*':
            # Zdejmujemy automat ze stosu
            nfa1 = nfaStack.pop() 

            # Tworzymy stan początkowy i końcowy dla nowego automatu
            initial = state()
            accept = state()

            # Łączymy krawędzie nowego stanu początkowego ze stanem początkowym nfa1 oraz nowym stanem akceptującym
            initial.edge1 = nfa1.initial
            initial.edge2 = accept

            # Łączymy krawędzie stanu akceptującego nfa1 ze stanem początkowym nfa1 oraz nowym stanem akceptującym
            nfa1.accept.edge1 = nfa1.initial
            nfa1.accept.edge2 = accept

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(initial, accept))

        elif c == '@': # konkatenacja
            # Zdejmujemy dwa automaty ze stosu
            nfa2 = nfaStack.pop() 
            nfa1 = nfaStack.pop()
            
            # Łączymy stan akceptujący pierwszego automatu ze stanem początkowym drugiego
            nfa1.accept.edge1 = nfa2.initial

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(nfa1.initial, nfa2.accept))
        elif c == '|': # alternatywa
            # Zdejmujemy dwa automaty ze stosu
            nfa2 = nfaStack.pop() 
            nfa1 = nfaStack.pop()

            # Tworzymy nowy stan początkowy i łączymy go ze stanami początkowymi nfa1 i nfa2
            initial = state()
            initial.edge1 = nfa1.initial
            initial.edge2 = nfa2.initial
            
            # Tworzymy nowy stan akceptujący i łączymy z nim stany akceptujące nfa1 i nfa2
            accept = state()
            nfa1.accept.edge1 = accept
            nfa2.accept.edge1 = accept

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(initial, accept))
        else:
            # Tworzymy stan początkowy i końcowy dla nowego automatu
            accept = state()
            initial = state()

            # Łączymy stan początkowy i akceptujący używając danego znaku ('c')
            initial.label = c 
            initial.edge1 = accept 

            # Nowy automat dodajemy na stos
            nfaStack.append(nfa(initial, accept))
    
    # Gotowy automat jest wierzchołkiem stosu (stos zawiera tylko 1 element)
    return nfaStack.pop()

W tym momencie nie za bardzo mamy jak przetestować nasz automat (potrzebna jest funkcja przechodząca po automacie). Potrzebne funkcje widoczne są poniżej.

In [267]:
# funkcja zwracająca zbiór osiągalnych stanów z danego stanu przechodząc po krawędziach 'epsilon'
def followEpsilons(state):
    # tworzenie zbioru stanów osiągalnych, początkowo zawiera tylko dany stan
    states = set()
    states.add(state)

    # Sprawdzamy czy stan ma krawędzie 'epsilon'
    if state.label is None:
        # Sprawdzenie czy edge1 jest stanem
        if state.edge1 is not None:
            # Jeśli tak, to przechodzimy dalej
            states |= followEpsilons(state.edge1)
        # Sprawdzenie czy edge2 jest stanem
        if state.edge2 is not None:
            # Jeśli tak, to przechodzimy dalej
            states |= followEpsilons(state.edge2)

    # Zwracamy zbiór osiągalnych stanów
    return states

# funkcja sprawdzająca zgodność słowa z danym wyrażeniem regularnym
# True - zgodne, False - niezgodne
def match(infix, string):
    # Zmieniamy postać infiksową na postfiksową i tworzymy nasz automat (nfa)
    postfix = to_postfix(infix)
    nfa = compile(postfix)

    # Zmienne symbolizujące teraźniejszy zbiór stanów i kolejny zbiór stanów
    currentState = set()
    nextState = set()

    # Początkowy zbiór stanów
    currentState |= followEpsilons(nfa.initial)

    # Przechodzimy po każdym znaku
    for s in string:
        # Przeszukujemy teraźniejszy zbiór stanów
        for c in currentState:
            # Sprawdzamy czy istnieje odpowiednia krawędź
            if c.label == s or c.label == '.':
                # Jeśli tak, dodajemy kolejne stany
                nextState |= followEpsilons(c.edge1)

        # Zamieniamy zbiory i resetujemy
        currentState = nextState
        nextState = set()
    
    # Zwracamy informacje czy automat zakończył pracę w stanie akceptującym
    return nfa.accept in currentState

Całe testowanie działania powyższego algorytmu widoczne jest w kolejnym punkcie. W tym miejscu chciałbym opisać możliwości zbudowanego automatu.  
  
Zbudowany automat pozwala interpretować wyrażenia regularne zawierające:  
 - litery ($[a-z],[A-Z]$), cyfry ($[0-9]$) oraz spacje 
 - kropkę $.$ symbolizującą dowolny znak
 - symbol gwiazdki $*$ symbolizujący powtórzenie znaku $0$ lub więcej razy  
 - symbol plusa $+$ symbolizujący powtórzenie znaku $1$ lub więcej razy
 - symbol pytajnika $?$ symbolizujący powtórzenie znaku $0$ lub $1$ raz
 - odpowiednio zagnieżdzone nawiasy $($ oraz $)$ umożliwiające użycie $*$, $+$ oraz $?$ dla więcej niż jednego znaku
 - wyrażenia w kwadratowych nawiasach oznaczające wystąpienie jednego z podanych znaków (np. $[abc]$ oznacza symbol $a$, $b$ lub $c$)
 - wyrażenia \s, \d, oraz \w oznaczające odpowiednio dowolny znak biały, cyfrę lub literę

Automat zakłada, iż wprowadzone wyrażenie jest poprawne.

# 2. Testowanie działania algorytmu

Przetestuję teraz działanie wszystkich wymienionych powyżej opcji. Na koniec przetestuje działanie kilku złożonych wyrażeń. Każdy wynik porównuje z Pythonowym regex'em.

In [247]:
import re

In [248]:
# ułatwienie
def python_regex(regex, txt):
    res = re.compile(regex).match(txt)
    if res is None:
        return False
    return True

### Litery, cyfry i spacje

In [249]:
reg = 'algorytm 20'
pyth_reg = reg + '$'
txt1 = ''
txt2 = 'algorytm 20'
txt3 = 'algorytm 201'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

False
False
True
True
False
False


### Kropka

In [250]:
reg = 'abc.'
pyth_reg = reg + '$'
txt1 = ''
txt2 = 'abcd'
txt3 = 'abcde'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

False
False
True
True
False
False


### Gwiazdka

In [251]:
reg = 'abc*'
pyth_reg = reg + '$'
txt1 = 'a'
txt2 = 'ab'
txt3 = 'abc'
txt4 = 'abccccccccccc'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

False
False
True
True
True
True
True
True


### Plus

In [252]:
reg = 'abc+'
pyth_reg = reg + '$'
txt1 = 'a'
txt2 = 'ab'
txt3 = 'abc'
txt4 = 'abccccccccccc'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

False
False
False
False
True
True
True
True


### Pytajnik

In [253]:
reg = 'abc?'
pyth_reg = reg + '$'
txt1 = 'a'
txt2 = 'ab'
txt3 = 'abc'
txt4 = 'abccccccccccc'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

False
False
True
True
True
True
False
False


### Nawiasy (oraz alternatywa)

In [259]:
reg = '(a|b|c)+'
pyth_reg = reg + '$'
txt1 = ''
txt2 = 'ab'
txt3 = 'abc'
txt4 = 'abcccccccccccd'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

False
False
True
True
True
True
False
False


### Kwadratowe nawiasy

In [260]:
reg = 'ab[cde]*'
pyth_reg = reg + '$'
txt1 = 'ab'
txt2 = 'abc'
txt3 = 'abcdecde'
txt4 = 'abcf'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

True
True
True
True
True
True
False
False


### Grupy znaków

In [261]:
reg = '(\w)+(\s)*(\d)*'
pyth_reg = reg + '$'
txt1 = ''
txt2 = 'alphabet'
txt3 = 'alphabet100'
txt4 = 'alphabet 100'
txt5 = 'alphabet '

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

print(match(reg,txt5))
print(python_regex(pyth_reg,txt5))

False
False
True
True
True
True
True
True
True
True


Jak widać wszystko działa zgodnie z założeniami. Wykonane zadanie jest zatem poprawne. Poniżej kilka ciekawszych wyrażeń regularnych.

### Numer telefonu

In [262]:
reg = '\d\d\d\d\d\d\d\d\d'
pyth_reg = reg + '$'
txt1 = '123'
txt2 = '123456789'
txt3 = '+48123432829'
txt4 = '794084017'
txt5 = 'phone number'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

print(match(reg,txt5))
print(python_regex(pyth_reg,txt5))

False
False
True
True
False
False
True
True
False
False


### Strona internetowa

In [263]:
reg = 'www.\w+.com'
pyth_reg = reg + '$'
txt1 = 'www'
txt2 = 'www.facebook.com'
txt3 = 'www.youtube.co'
txt4 = 'localhost:8882'
txt5 = 'www.ilovealgorithms.com'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

print(match(reg,txt5))
print(python_regex(pyth_reg,txt5))

False
False
True
True
False
False
False
False
True
True


### Liczby rzeczywiste

In [265]:
reg = '\d*.\d+'
pyth_reg = reg + '$'
txt1 = ''
txt2 = '512.321'
txt3 = '100.000'
txt4 = '.123'
txt5 = '100.000.000'

print(match(reg,txt1))
print(python_regex(pyth_reg,txt1))

print(match(reg,txt2))
print(python_regex(pyth_reg,txt2))

print(match(reg,txt3))
print(python_regex(pyth_reg,txt3))

print(match(reg,txt4))
print(python_regex(pyth_reg,txt4))

print(match(reg,txt5))
print(python_regex(pyth_reg,txt5))

False
False
True
True
True
True
True
True
False
False


# 3. Wniosek

Wykonane zadanie było zdecydowanie jednym z ciekawszych na tym kursie. Wykorzystany tutaj algorytm jest powszechnie używany w wielu miejscach (np. poprawność hasła, maila itp) więc implementacja takiego algorytmu jest rzeczą jak najbardziej pożyteczną (jak i satysfakcjonującą). Poprzedni punkt pokazuje poprawność wykonanego zadania. Stworzony silnik warto poszerzyć o dodatkowe możliwości (np. dodanie przedziału przy pomocy myślnika $[a-z]$).